In [2]:
import pandas as pd
import sys
import os
# sys.path.append(os.path.abspath('..'))

In [3]:
df=pd.read_csv('../Data/data.csv')

feature engineering the result should here that contain in the columnets  that are added becaus of feature enfinering like 


In [4]:
# sys.path.append(os.path.abspath('../script'))
# from feature_engineering import feature_engineering

# fdf=feature_engineering(df)

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer

def calculate_woe_iv(data, feature, target):
    """
    Calculate Weight of Evidence (WOE) and Information Value (IV) for a given feature.

    Parameters:
    data (pd.DataFrame): Input DataFrame
    feature (str): Feature column name
    target (str): Target column name

    Returns:
    pd.DataFrame: DataFrame with WOE values for the feature
    """
    eps = 1e-7  # Small value to avoid division by zero
    grouped = data.groupby(feature)[target].agg(['count', 'sum'])
    grouped['non_event'] = grouped['count'] - grouped['sum']
    grouped['event_rate'] = grouped['sum'] / grouped['sum'].sum()
    grouped['non_event_rate'] = grouped['non_event'] / grouped['non_event'].sum()
    grouped['woe'] = np.log((grouped['event_rate'] + eps) / (grouped['non_event_rate'] + eps))
    grouped['iv'] = (grouped['event_rate'] - grouped['non_event_rate']) * grouped['woe']
    return grouped[['woe', 'iv']]

def feature_engineering(df):
    """
    Perform feature engineering on the given DataFrame.

    Parameters:
    df (pd.DataFrame): Input DataFrame

    Returns:
    pd.DataFrame: DataFrame with engineered features.
    """

    # Create Aggregate Features
    print("\n--- Creating Aggregate Features ---")
    df['TotalTransactionAmount'] = df.groupby('CustomerId')['Amount'].transform('sum')
    df['AvgTransactionAmount'] = df.groupby('CustomerId')['Amount'].transform('mean')
    df['TransactionCount'] = df.groupby('CustomerId')['Amount'].transform('count')
    df['StdTransactionAmount'] = df.groupby('CustomerId')['Amount'].transform('std')

    # Extract Date-Based Features
    print("\n--- Extracting Date-Based Features ---")
    df['TransactionHour'] = pd.to_datetime(df['TransactionStartTime']).dt.hour
    df['TransactionDay'] = pd.to_datetime(df['TransactionStartTime']).dt.day
    df['TransactionMonth'] = pd.to_datetime(df['TransactionStartTime']).dt.month
    df['TransactionYear'] = pd.to_datetime(df['TransactionStartTime']).dt.year

    # Encode Categorical Variables
    print("\n--- Encoding Categorical Variables ---")
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    high_cardinality_cols = [col for col in categorical_cols if df[col].nunique() > 50]
    low_cardinality_cols = [col for col in categorical_cols if df[col].nunique() <= 50]

    # One-Hot Encoding for low cardinality columns
    if low_cardinality_cols:
        one_hot_encoder = OneHotEncoder(sparse_output=False, drop='first')
        one_hot_encoded = pd.DataFrame(one_hot_encoder.fit_transform(df[low_cardinality_cols]),
                                       columns=one_hot_encoder.get_feature_names_out(low_cardinality_cols))
        df = pd.concat([df, one_hot_encoded], axis=1)

    # Label Encoding for high cardinality columns
    for col in high_cardinality_cols:
        df[f'{col}_Encoded'] = LabelEncoder().fit_transform(df[col])

    # Handle Missing Values
    print("\n--- Handling Missing Values ---")
    imputer = SimpleImputer(strategy='mean')  # Example with mean, change strategy as needed
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    df[numerical_cols] = imputer.fit_transform(df[numerical_cols])

    # Normalize/Standardize Numerical Features
    print("\n--- Normalizing/Standardizing Numerical Features ---")
    scaler = MinMaxScaler()  # Use StandardScaler() for standardization
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

    # Feature Engineering with Weight of Evidence (WOE)
    print("\n--- Feature Engineering with WOE ---")
    if 'FraudResult' in df.columns:
        for feature in categorical_cols:
            woe_iv = calculate_woe_iv(df, feature, 'FraudResult')
            df = df.merge(woe_iv['woe'], how='left', left_on=feature, right_index=True, suffixes=('', f'_WOE_{feature}'))

    print("Feature Engineering completed successfully!")
    return df
feature_engineering(df)


--- Creating Aggregate Features ---

--- Extracting Date-Based Features ---

--- Encoding Categorical Variables ---

--- Handling Missing Values ---

--- Normalizing/Standardizing Numerical Features ---

--- Feature Engineering with WOE ---
Feature Engineering completed successfully!


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,...,woe_WOE_BatchId,woe_WOE_AccountId,woe_WOE_SubscriptionId,woe_WOE_CustomerId,woe_WOE_CurrencyCode,woe_WOE_ProviderId,woe_WOE_ProductId,woe_WOE_ProductCategory,woe_WOE_ChannelId,woe_WOE_TransactionStartTime
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,0.0,ProviderId_6,ProductId_10,airtime,...,-4.66104,-8.841338,-8.841338,-9.430743,0.0,-3.136999,-1.645064,-1.620378,0.472374,-4.66104
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,0.0,ProviderId_4,ProductId_6,financial_services,...,-4.66104,-4.134367,-4.189071,-9.430743,0.0,-2.736864,-4.189224,0.565446,-2.709034,-4.66104
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,0.0,ProviderId_6,ProductId_1,airtime,...,-4.66104,-5.349448,-5.349448,-5.349448,0.0,-3.136999,-12.618822,-1.620378,0.472374,-4.66104
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,0.0,ProviderId_1,ProductId_21,utility_bill,...,-4.66104,-7.910002,-7.910002,-8.289376,0.0,1.618898,0.271620,1.134959,0.472374,-4.66104
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,0.0,ProviderId_4,ProductId_6,financial_services,...,-4.66104,-4.134367,-4.189071,-8.289376,0.0,-2.736864,-4.189224,0.565446,-2.709034,-4.66104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95657,TransactionId_89881,BatchId_96668,AccountId_4841,SubscriptionId_3829,CustomerId_3078,UGX,0.0,ProviderId_4,ProductId_6,financial_services,...,-4.66104,-4.134367,-4.189071,-11.002441,0.0,-2.736864,-4.189224,0.565446,-2.709034,-4.66104
95658,TransactionId_91597,BatchId_3503,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,0.0,ProviderId_6,ProductId_10,airtime,...,-4.66104,-7.947729,-7.947729,-8.412961,0.0,-3.136999,-1.645064,-1.620378,0.472374,-4.66104
95659,TransactionId_82501,BatchId_118602,AccountId_4841,SubscriptionId_3829,CustomerId_3874,UGX,0.0,ProviderId_4,ProductId_6,financial_services,...,-4.66104,-4.134367,-4.189071,-8.412961,0.0,-2.736864,-4.189224,0.565446,-2.709034,-4.66104
95660,TransactionId_136354,BatchId_70924,AccountId_1346,SubscriptionId_652,CustomerId_1709,UGX,0.0,ProviderId_6,ProductId_19,tv,...,-4.66104,-10.509499,-10.509499,-10.913049,0.0,-3.136999,-11.523677,-11.805380,0.472374,-4.66104
